# Unraveling Gravitational Ripples: Neural Network Classification
### GASF vs FFT vs QTransform with New Data
### **GWGASF Members:** $Daniel\;Fredin^{1,2}$, $Cole\;Welch^{1,3}$, $Chia-Jui\;Chou^{4}$, $Andy\;Chen^{4}$, & $Shih-Chieh\;Hsu^{1}$
#### $^{1}University\;of\;Washington,\;Seattle\;USA$; $^{2}University\;of\;Massachusetts\;Lowell,\;Lowell\;USA$;$^{3}Georgia\;Institute\;of\;Technology,\;Atlanta\;USA$; $^{4}National\;Yang\;Ming\;Chiao\;Tung\;University,\;Hsinchu\;City,\;TW$

--------------------

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torch import nn
from pyts.image import GramianAngularField
from tqdm import tqdm
import os


from gasf.utils import h5_thang
# import random

# import h5py as h5
# from gasf.sampling import # Finish module import
# import ml4gw

# Initializes numpy and pytorch random seeds for reproducibility 
seed = 55
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

# Preparing the data

##### need to modularize

In [ ]:
### ------------- Load dataset ------------- ###


# Loading datafiles and only load a slice of the total data.
numSamples = 1200 # from each glitch/signal/background

# bbh signals
bbh_signals_filename = '/home/dfredin/gwgasf/data/raw_data/bbh_dataset_p1.hdf5'
bbh_info = h5_thang(bbh_signals_filename)
bbh_keys = bbh_info.h5_keys()
H1_bbh = bbh_info.h5_data()['H1'][0:numSamples]
L1_bbh = bbh_info.h5_data()['L1'][0:numSamples]

# H1 background
H1_bg_filename = '/home/dfredin/gwgasf/data/raw_data/H1_bg_dataset_p1.hdf5'
H1_bg_info = h5_thang(H1_bg_filename)
H1_bg_keys = H1_bg_info.h5_keys()
H1_bg = H1_bg_info.h5_data()['background_noise'][0:numSamples]
# L1 background
L1_bg_filename = '/home/dfredin/gwgasf/data/raw_data/L1_bg_dataset_p1.hdf5'
L1_bg_info = h5_thang(L1_bg_filename)
L1_bg_keys = L1_bg_info.h5_keys()
L1_bg = L1_bg_info.h5_data()['background_noise'][0:numSamples]

# H1 glitch
H1_glitch_filename = '/home/dfredin/gwgasf/data/raw_data/H1_glitch_dataset_p1.hdf5'
H1_glitch_info = h5_thang(H1_glitch_filename)
H1_glitch_keys = H1_glitch_info.h5_keys()
H1_glitch = H1_glitch_info.h5_data()['glitch'][0:numSamples]
# L1 glitch
L1_glitch_filename = '/home/dfredin/gwgasf/data/raw_data/L1_glitch_dataset_p1.hdf5'
L1_glitch_info = h5_thang(L1_glitch_filename)
L1_glitch_keys = L1_glitch_info.h5_keys()
L1_glitch = L1_glitch_info.h5_data()['glitch'][0:numSamples]

# # hfsg signals
# hfsg_signals_filename = '/home/dfredin/gwgasf/data/raw_data/hfsg_dataset_p1.hdf5'
# hfsg_info = h5_thang(hfsg_signals_filename)
# hfsg_keys = hfsg_info.h5_keys()
# H1_hfsg = hfsg_info.h5_data()['H1'][0:numSamples]
# L1_hfsg = hfsg_info.h5_data()['L1'][0:numSamples]
# # lfsg signals
# lfsg_signals_filename = '/home/dfredin/gwgasf/data/raw_data/lfsg_dataset_p1.hdf5'
# lfsg_info = h5_thang(lfsg_signals_filename)
# lfsg_keys = lfsg_info.h5_keys()
# H1_lfsg = lfsg_info.h5_data()['H1'][0:numSamples]
# L1_lfsg = lfsg_info.h5_data()['L1'][0:numSamples]

# variables = [H1_bbh, L1_bbh, H1_bg, H1_glitch, L1_bg, L1_glitch, H1_hfsg, L1_hfsg, H1_lfsg, L1_lfsg]
# names = ['H1 BBH', 'L1 BBH', 'H1 BG', 'H1 Glitch', 'L1 BG', 'L1 Glitch', 'H1 HFSG', 'L1 HFSG', 'H1 LFSG', 'L1 LFSG']

# for name, var in zip(names, variables):
#     print(f"{name} Shape: {var.shape}")

variables = [H1_bbh, L1_bbh, H1_bg, H1_glitch, L1_bg, L1_glitch]
names = ['H1 BBH', 'L1 BBH', 'H1 BG', 'H1 Glitch', 'L1 BG', 'L1 Glitch']

for name, var in zip(names, variables):
    print(f"{name} Shape: {var.shape}")

In [ ]:
### ------------- HIGH SNR GLITCH ------------- ###

# This code block filters glitch signals based on their signal-to-noise ratio (SNR) 
H1_glitch_snr = H1_glitch_info.h5_data()['glitch_info']
L1_glitch_snr = L1_glitch_info.h5_data()['glitch_info']

# Filter glitch signals with SNR >= 10 for H1 detector
h1_indices = np.where(H1_glitch_snr['snr'] >= 15)
H1_glitch_snr_high = H1_glitch_info.h5_data()['glitch'][h1_indices]

# Filter glitch signals with SNR >= 10 for L1 detector
l1_indices = np.where(L1_glitch_snr['snr'] >= 15)
L1_glitch_snr_high = L1_glitch_info.h5_data()['glitch'][l1_indices]

# Truncate the glitch signals to have the same length
H1_glitch_snr_high = H1_glitch_snr_high[0:len(L1_glitch_snr_high)]
L1_glitch_snr_high = L1_glitch_snr_high[0:len(L1_glitch_snr_high)]

# Create a stack of high SNR glitch signals from both detectors
glitch_highsnr_signals = np.dstack((H1_glitch_snr_high, L1_glitch_snr_high))

# Assign the stack of glitch signals to the final variable
glitch_signals = glitch_highsnr_signals

# Clean up variables to free up memory
del L1_glitch_snr_high, H1_glitch_snr_high, H1_glitch_snr, L1_glitch_snr, glitch_highsnr_signals

In [ ]:
### ------------- STACK ARRAYS ------------- ###

## Stacking the arrays for the H1 and L1 detectors

bbh_signals = np.dstack((H1_bbh, L1_bbh))
bg_signals = np.dstack((H1_bg, L1_bg))
# TODO: Add the other high SNR glitches
# glitch_signals = np.dstack((H1_glitch, L1_glitch))
# hfsg_signals = np.dstack((H1_hfsg, L1_hfsg))
# lfsg_signals = np.dstack((H1_lfsg, L1_lfsg))

del H1_bbh, L1_bbh, H1_bg, L1_bg, H1_glitch, L1_glitch

# variables = [bbh_signals, bg_signals, glitch_signals, hfsg_signals, lfsg_signals]
# names = ['BBH Signals', 'BG Signals', 'Glitch Signals', 'HFSG Signals', 'LFSG Signals']

# for name, var in zip(names, variables):
#     print(f"{name} Shape: {var.shape}")

x_train = np.concatenate((glitch_signals, bbh_signals, bg_signals))
x_train.shape
# x_train = np.concatenate((glitch_train, bbh_train, overlap_train), axis=0).transpose((0,2,1))


In [ ]:
## ---------------- ONE-HOT ENCODING ----------------------------##

# NOTE: 
# Multiclassifier
anomaly_class = {
    'Glitch': [1, 0, 0],
    'Signal': [0, 1, 0],
    'Background': [0, 0, 1]
}

glitch_train_ids = np.full((glitch_signals.shape[0], 3), anomaly_class['Glitch'])
bbh_train_ids = np.full((bbh_signals.shape[0], 3), anomaly_class['Signal'])
bg_train_ids = np.full((bg_signals.shape[0], 3), anomaly_class['Background'])


y_train_ids = np.concatenate((glitch_train_ids, bbh_train_ids, bg_train_ids), axis=0)
del bbh_signals, bg_signals, glitch_signals
del glitch_train_ids, bbh_train_ids, bg_train_ids

In [ ]:
n_train = 1000

# Generate the idx_train, idx_val to randomly sample from the training set and validation set.
idx_train = np.random.randint(0, len(x_train), size=n_train)

y_train = y_train_ids[idx_train]

### ------------- GASF CONVERSION ------------- ###

GASF = GramianAngularField(image_size=194, sample_range = (-1,1), method="summation")
# Convert training data to image format
img_x_train_dec1 = GASF.transform(x_train[idx_train,:,0])
img_x_train_dec2 = GASF.transform(x_train[idx_train,:,1])

img_x_train = np.stack((img_x_train_dec1, img_x_train_dec2), axis=1)
del img_x_train_dec1, img_x_train_dec2


In [ ]:
### ------------- Split into training and validation datasets ------------- ###

# Assuming img_x_train is defined and has the shape (N, 2, 194, 194)
# Assuming y_train is defined and has the shape (N, 3), where N can be any integer

# Total number of samples
total_samples = img_x_train.shape[0]

# Calculate the number of samples in each dataset
## NOTE: MAYBE USER INPUT FOR SPLIT %???
num_train = int(0.8 * total_samples)  # 80% for training
num_test = int(0.15 * total_samples)  # 15% for testing
num_val = total_samples - num_train - num_test  # Remaining 5% for validation, adjusted to avoid rounding issues

# Create training, testing, and validation datasets for img_x_train
x_train_data = img_x_train[:num_train]
x_test_data = img_x_train[num_train:num_train + num_test]
x_val_data = img_x_train[num_train + num_test:]

# Create training, testing, and validation datasets for y_train
y_train_data = y_train[:num_train]
y_test_data = y_train[num_train:num_train + num_test]
y_val_data = y_train[num_train + num_test:]

# Print shapes to verify the distribution
strains = [x_train_data, x_test_data, x_val_data]
targets = [y_train_data, y_test_data, y_val_data]
labels = ['Training', 'Testing', 'Validation']

# Collecting and printing the shape of each array
array_shapes = [arr.shape for arr in strains]
for i, shape in enumerate(array_shapes):
    print(f"{labels[i]} Data Shape: {shape}")

array_shapes = [arr.shape for arr in targets]
for i, shape in enumerate(array_shapes):
    print(f"{labels[i]} Targets Shape: {shape}")


# Test Plots

In [ ]:
# Glitch Plot

y_train_label = str(y_train[0])
# Create a new figure for the glitch plot
plt.figure(figsize=(10, 6))
plt.plot(x_train[idx_train, :, 0][0])  # Adjusted the indexing based on typical data structure
plt.title(f'Label: {y_train_label}')
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()

# Create a new figure for the glitch GASF plot
plt.figure(figsize=(10, 6))
plt.imshow(img_x_train[:,0,:,:][0], cmap='rainbow', origin='lower')
plt.title(f'Label: {y_train_label}')
plt.show()


# Signal Plot

y_train_label = str(y_train[548])
# Create a new figure for the signal plot
plt.figure(figsize=(10, 6))
plt.plot(x_train[idx_train,:,0][548])
plt.title(f'Label: {y_train_label}')
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()

# Create a new figure for the signal GASF plot
plt.figure(figsize=(10, 6))
plt.imshow(img_x_train[:,0,:,:][548], cmap='rainbow', origin='lower')
plt.title(f'Label: {y_train_label}')
plt.show()


# Background Plot

y_train_label = str(y_train[55])
# Create a new figure for the Background plot
plt.figure(figsize=(10, 6))
plt.plot(x_train[idx_train,:,0][55])
plt.title(f'Label: {y_train_label}')
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()

# Create a new figure for the Background GASF plot
plt.figure(figsize=(10, 6))
plt.imshow(img_x_train[:,0,:,:][55], cmap='rainbow', origin='lower')
plt.title(f'Label: {y_train_label}')
plt.show()

# DataLoader

In [ ]:
class MyDataset(Dataset):

    def __init__(self, data, target, ifos=3, kernel_size=4096):

        # Get data type from https://pytorch.org/docs/stable/tensors.html
        # self.data = torch.FloatTensor(
        #     data.reshape([-1, ifos, kernel_size])
        # )
        # 
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(target)
    
    def __len__(self):
        
        return len(self.data)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y

In [ ]:
# Load Training data into pytorch tensor
dataset = MyDataset(strains[0], targets[0])
training_data = DataLoader(
    dataset, 
    batch_size=32, 
    shuffle=True
)

# Load testing data into pytorch tensor
dataset_t = MyDataset(strains[2], targets[2])
testing_data = DataLoader(
    dataset_t, 
    batch_size=32, 
    shuffle=True
)


# Load Validation data into pytorch tensor
dataset_v = MyDataset(strains[1], targets[1])
validation_data = DataLoader(
    dataset_v, 
    batch_size=32, 
    shuffle=True
)

In [ ]:
strains[0].shape

# CNN Model

In [ ]:

               ## # | 64 x 64 | 127 x 127 | 194 x 194 |
cnn1In = 2     ## # |    2    |     2     |     2     |
cnn1Out = 64   ## # |    16   |     16    |     16    |    
cnn1K = 6      ## # |    4    |     5     |     6     |
cnn1S = 2      ## # |    2    |     2     |     2     |
cnn1P = 1      ## # |    1    |     2     |     1     |
batch1 = 64    ## # |    16   |     16    |     16    |
max1K = 5      ## # |    4    |     2     |     5     |
max1S = 3      ## # |    2    |     2     |     3     |
max1P = 1      ## # |    1    |     0     |     1     |

cnn2In = 64    ## # |    16   |     16    |     16    |
cnn2Out = 128   ## # |    32   |     32    |     32    |
cnn2K = 4      ## # |    3    |     4     |     4     |
cnn2S = 2      ## # |    1    |     2     |     2     |
cnn2P = 1      ## # |    1    |     1     |     1     |
batch2 = 128    ## # |    32   |     32    |     32    |
max2K = 2      ## # |    2    |     2     |     2     |
max2S = 2      ## # |    2    |     2     |     2     |
max2P = 0      ## # |    0    |     0     |     0     |

### -------------194 x 194 Model Definition------------- ###

class CNNModel(nn.Module):
    
    def __init__(self):
        
        super(CNNModel, self).__init__()

                # First convolution layer (2 channel -> 16 channels, preserve original dimension by adding padding = 2) 
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=cnn1Out, kernel_size=cnn1K, stride=cnn1S, padding=cnn1P),
            nn.ReLU(),
            nn.BatchNorm2d(batch1),
            nn.MaxPool2d(kernel_size=max1K, stride=max1S, padding=max1P)
            )
        
        # Second convolution layer (16 channel -> 32 channels, preserve dimension by adding padding = 2)
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=cnn2In, out_channels=cnn2Out, kernel_size=cnn2K, stride=cnn2S, padding=cnn2P),
            nn.ReLU(),
            nn.BatchNorm2d(batch2),
            nn.MaxPool2d(kernel_size=max2K, stride=max2S, padding=max2P)
            )

        # Fully connected layer that takes the flattened output of layer 2 (32, 8 ,8) -> (2048) and output 2 classes
        self.fc1 = nn.Linear(in_features=128*8*8, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=3)
        self.dropcnn = nn.Dropout(0.25)
        self.dropfc = nn.Dropout(0.6)

    
    def forward(self, x):

        # input image -> conv -> batchnorm -> relu -> maxpool
        out = self.layer1(x)
        out = self.dropcnn(out)
        # 1st maxpool output -> conv -> batchnorm -> relu -> maxpool
        out = self.layer2(out)
        out = self.dropcnn(out)
        # flatten the second maxpool output to be used as input into FCN layer
        out = out.view(out.size(0), -1)

        # Pass flattened output into FCN layers
        out = self.fc1(out)
        out = self.dropfc(out)
        out = self.fc2(out)
        out = self.dropfc(out)
        out = self.fc3(out)
        
        return out

## Hyperparameters

In [ ]:
  ### -------------Hyperparameters------------- ###

# Initialize the CNN model
GASF_Model = CNNModel().to(device)

# An easy way to give you the outline of your model
summary(GASF_Model,  (2, 194, 194), len(strains[0]))

learning_rate = 0.0005     # 0.00005
epochs = 25                 # 40
# batchsize = 225             # 768
L2_reg = 0.001            # 0.00005

# Define loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    GASF_Model.parameters(), 
    lr = learning_rate, 
    weight_decay=L2_reg)

# Training

In [ ]:
cost_value = np.empty(epochs)
cost_valid_value = np.empty(epochs)

for i in range(epochs):
    t_cost = 0
    for j, (x, y) in enumerate(tqdm(training_data, desc=f'Epoch {i+1}')):
        x = x.to(device)
        y = y.to(device)
        # print(y)

        p_value = GASF_Model(x)

        p_value = torch.transpose(p_value,0,1)
        y = torch.transpose(y,0,1)
        # print(p_value)
        y = torch.argmax(y, dim = 1)
        cost = loss_func(p_value, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        t_cost += cost.item()

    cost_value[i] = t_cost/(j+1)
        
    print('========')
    print(f'Cost{round(cost_value[i], 2)}')
    print('========', '\n')

    with torch.no_grad():
        v_cost = 0
        for run, (a, b) in enumerate(validation_data):
            a = a.to(device)
            b = b.to(device)
            
            p_value = GASF_Model(a)

            p_value = torch.transpose(p_value,0,1)
            b = torch.transpose(b,0,1)

            b = torch.argmax(b, dim = 1)
            cost_valid = loss_func(p_value, b)
            
            v_cost += cost_valid.item()
            
        cost_valid_value[i] = v_cost/(run+1)
        
        print('===================')
        print(f'Validation Cost{round(cost_valid_value[i], 2)}')
        print('===================', '\n')



# Save the model
model_save_path='/home/dfredin/gwgasf/models/gasf_model.pth'
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
torch.save(GASF_Model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')

# torch.save(GASF_Model, model_save_path)


In [ ]:
total_epoch = np.linspace(1, epochs, epochs)
plt.title('Loss value to epoch')
plt.plot(total_epoch, cost_value, label = 'Training Set')
plt.plot(total_epoch, cost_valid_value, label = 'Validation Set')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()
train_val_Lossplot_filename = '/home/dfredin/gwgasf/results/figures/train_val_Loss.png'
plt.savefig(train_val_Lossplot_filename, bbox_inches = 'tight')
plt.show()
plt.close()

-------

## Confusion Matrix Analysis

In [ ]:
# torch tensor device check

if device.type == 'cpu':
    dtype = torch.FloatTensor
elif device.type =='cuda':
    dtype = torch.cuda.FloatTensor

In [ ]:
def calculate_metrics(conf_matrix):
    # Calculate precision, recall, and F1 score for the entire model
    TP = np.sum(np.diag(conf_matrix))  # True Positives
    FP = np.sum(np.sum(conf_matrix, axis=0) - np.diag(conf_matrix))  # False Positives
    FN = np.sum(np.sum(conf_matrix, axis=1) - np.diag(conf_matrix))  # False Negatives

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

def plot_confusion_matrix(data, title):
    num_classes = 3
    conf_matrix = torch.zeros([num_classes,num_classes]).to(device)
    num_count = torch.zeros([num_classes]).to(device)

    with torch.no_grad():
        for num, (x, y) in enumerate(tqdm(data)):
            Real_label = y.to(device)

            # Convert sofmax output to onehot
            max_class = torch.Tensor.argmax(GASF_Model(x.to(device)), axis=1)
            pred = F.one_hot(max_class)

            # Accumlating statical value
            conf_matrix += torch.matmul(pred.T.type(dtype), Real_label.type(dtype))
            num_count += Real_label.sum(axis=0)

        print(num_count)
        num_count = num_count.detach().cpu().numpy().astype('float64')
        conf_matrix = conf_matrix.detach().cpu().numpy().astype('float64')

    # Normalize the confusion matrix.
    conf_matrix /= num_count

    precision, recall, f1_score = calculate_metrics(conf_matrix)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1_score}')

    # Labels for a 3x3 confusion matrix
    lab = np.array([['True Positive', 'False Positive', 'False Positive'], 
                    ['False Negative', 'True Negative', 'False Positive'],
                    ['False Negative', 'False Negative', 'True Negative']])

    plt.figure()
    color = plt.pcolormesh([conf_matrix[2], conf_matrix[1], conf_matrix[0]], cmap = 'Wistia', vmin=0, vmax=1)

    # Adjust the loop for a 3x3 matrix
    for i in range(3):
        for j in range(3):
            plt.text(i+.5, j+.5, 
                     f'{lab[2-j, i]}\n{round(conf_matrix[2-j, i], 3)}', 
                     ha='center', 
                     va='center')

    # Update the ticks for three classes
    plt.xticks([.5, 1.5, 2.5], ['Glitch', 'Signal', 'Background'])
    plt.yticks([.5, 1.5, 2.5], ['Background', 'Signal', 'Glitch'], rotation=45)
    plt.xlabel('Predicted Value')
    plt.ylabel('Actual Value')
    plt.title(f'3x3 {title} Confusion Matrix')
    plt.colorbar(color)
    plt.savefig(f'/home/dfredin/gwgasf/results/figures/{title}_confuMatrix.png', bbox_inches = 'tight')
    plt.show()
    plt.close()

# Call the function for each of your datasets
plot_confusion_matrix(validation_data, 'Validation')
plot_confusion_matrix(training_data, 'Training')
plot_confusion_matrix(testing_data, 'Test')